In [1]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

# Tạo DataFrame từ bảng student
data_student = {
    "student_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "name": ["Nguyen Minh Hoang", "Tran Thi Lan", "Pham Van Nam", "Le Thanh Huyen", "Vu Quoc Anh",
             "Dang Thuy Linh", "Bui Tien Dung", "Ho Ngoc Mai", "Duong Huu Phuc", "Cao Thi Hanh"],
    "class": ["May Tinh", "Kinh Te", "Toan Tin", "Toan Tin", "May Tinh",
              "May Tinh", "Kinh Te", "Toan Tin", "Kinh Te", "May Tinh"],
    "course_id": [12, 34, None, 20, 24, 24, 34, 20, None, None],
    "score": [6.7, 9.2, 7.9, 7.2, 8.0, 5.5, 9.2, 8.8, 7.2, 7.0]
}
df_student = pd.DataFrame(data_student)

# Tạo DataFrame từ bảng course
data_course = {
    "id": [12, 34, 26],
    "course_name": ["Giai tich", "Thong ke", "Tin hoc"]
}
df_course = pd.DataFrame(data_course)

# Kết nối SQLite
conn = sqlite3.connect(':memory:')
df_student.to_sql('student', conn, index=False, if_exists='replace')
df_course.to_sql('course', conn, index=False, if_exists='replace')

3

# 1. Kết nối hai bảng.

## a, Sử dụng Tích Descartes (Cross Join):

In [2]:
# Tích Descartes (Cross Join)
query_cross = '''
SELECT * FROM student CROSS JOIN course;
'''
df_cross = pd.read_sql(query_cross, conn)
print("\nTích Descartes (Cross Join) Result:")
print(df_cross)


Tích Descartes (Cross Join) Result:
    student_id               name     class  course_id  score  id course_name
0            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12   Giai tich
1            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  34    Thong ke
2            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  26     Tin hoc
3            2       Tran Thi Lan   Kinh Te       34.0    9.2  12   Giai tich
4            2       Tran Thi Lan   Kinh Te       34.0    9.2  34    Thong ke
5            2       Tran Thi Lan   Kinh Te       34.0    9.2  26     Tin hoc
6            3       Pham Van Nam  Toan Tin        NaN    7.9  12   Giai tich
7            3       Pham Van Nam  Toan Tin        NaN    7.9  34    Thong ke
8            3       Pham Van Nam  Toan Tin        NaN    7.9  26     Tin hoc
9            4     Le Thanh Huyen  Toan Tin       20.0    7.2  12   Giai tich
10           4     Le Thanh Huyen  Toan Tin       20.0    7.2  34    Thong ke
11           4     Le Thanh

## b, Sử dụng INNER JOIN , LEFT JOIN , RIGHT JOIN , FULL OUTER JOIN:

In [3]:
# INNER JOIN
query_inner = '''
SELECT s.student_id, s.name, s.class, s.course_id, c.course_name, s.score 
FROM student s 
INNER JOIN course c ON s.course_id = c.id
'''
df_inner = pd.read_sql(query_inner, conn)
print("INNER JOIN Result:")
print(df_inner)

INNER JOIN Result:
   student_id               name     class  course_id course_name  score
0           1  Nguyen Minh Hoang  May Tinh       12.0   Giai tich    6.7
1           2       Tran Thi Lan   Kinh Te       34.0    Thong ke    9.2
2           7      Bui Tien Dung   Kinh Te       34.0    Thong ke    9.2


In [4]:
# LEFT JOIN
query_left = '''
SELECT s.student_id, s.name, s.class, s.course_id, c.course_name, s.score 
FROM student s 
LEFT JOIN course c ON s.course_id = c.id
'''
df_left = pd.read_sql(query_left, conn)
print("\nLEFT JOIN Result:")
print(df_left)


LEFT JOIN Result:
   student_id               name     class  course_id course_name  score
0           1  Nguyen Minh Hoang  May Tinh       12.0   Giai tich    6.7
1           2       Tran Thi Lan   Kinh Te       34.0    Thong ke    9.2
2           3       Pham Van Nam  Toan Tin        NaN        None    7.9
3           4     Le Thanh Huyen  Toan Tin       20.0        None    7.2
4           5        Vu Quoc Anh  May Tinh       24.0        None    8.0
5           6     Dang Thuy Linh  May Tinh       24.0        None    5.5
6           7      Bui Tien Dung   Kinh Te       34.0    Thong ke    9.2
7           8        Ho Ngoc Mai  Toan Tin       20.0        None    8.8
8           9     Duong Huu Phuc   Kinh Te        NaN        None    7.2
9          10       Cao Thi Hanh  May Tinh        NaN        None    7.0


In [5]:
# RIGHT JOIN (dùng LEFT JOIN với đảo thứ tự bảng)
query_right = '''
SELECT s.student_id, s.name, s.class, s.course_id, c.course_name, s.score 
FROM course c 
LEFT JOIN student s ON s.course_id = c.id
'''
df_right = pd.read_sql(query_right, conn)
print("\nRIGHT JOIN Result:")
print(df_right)


RIGHT JOIN Result:
   student_id               name     class  course_id course_name  score
0         1.0  Nguyen Minh Hoang  May Tinh       12.0   Giai tich    6.7
1         2.0       Tran Thi Lan   Kinh Te       34.0    Thong ke    9.2
2         7.0      Bui Tien Dung   Kinh Te       34.0    Thong ke    9.2
3         NaN               None      None        NaN     Tin hoc    NaN


In [6]:
# FULL OUTER JOIN (kết hợp LEFT JOIN và RIGHT JOIN)
query_full_outer = '''
SELECT s.student_id, s.name, s.class, s.course_id, c.course_name, s.score 
FROM student s 
LEFT JOIN course c ON s.course_id = c.id
UNION
SELECT s.student_id, s.name, s.class, s.course_id, c.course_name, s.score 
FROM course c 
LEFT JOIN student s ON s.course_id = c.id
'''
df_full_outer = pd.read_sql(query_full_outer, conn)
print("\nFULL OUTER JOIN Result:")
print(df_full_outer)


FULL OUTER JOIN Result:
    student_id               name     class  course_id course_name  score
0          NaN               None      None        NaN     Tin hoc    NaN
1          1.0  Nguyen Minh Hoang  May Tinh       12.0   Giai tich    6.7
2          2.0       Tran Thi Lan   Kinh Te       34.0    Thong ke    9.2
3          3.0       Pham Van Nam  Toan Tin        NaN        None    7.9
4          4.0     Le Thanh Huyen  Toan Tin       20.0        None    7.2
5          5.0        Vu Quoc Anh  May Tinh       24.0        None    8.0
6          6.0     Dang Thuy Linh  May Tinh       24.0        None    5.5
7          7.0      Bui Tien Dung   Kinh Te       34.0    Thong ke    9.2
8          8.0        Ho Ngoc Mai  Toan Tin       20.0        None    8.8
9          9.0     Duong Huu Phuc   Kinh Te        NaN        None    7.2
10        10.0       Cao Thi Hanh  May Tinh        NaN        None    7.0


# 2. Tổng số , điểm TB , phân loại sinh viên.

In [7]:
# Cập nhật course_id bị thiếu
query_update = '''
UPDATE student 
SET course_id = (SELECT id FROM course WHERE student.course_id IS NULL LIMIT 1) 
WHERE course_id IS NULL;
'''
conn.execute(query_update)

## a, Tính tổng số sinh viên và điểm trung bình theo lớp:

In [8]:
query_avg_class = '''
SELECT class, COUNT(student_id) as total_students, AVG(score) as avg_score
FROM student 
GROUP BY class
'''
df_avg_class = pd.read_sql(query_avg_class, conn)
print("\nTổng số sinh viên và điểm trung bình theo lớp:")
print(df_avg_class)


Tổng số sinh viên và điểm trung bình theo lớp:
      class  total_students  avg_score
0   Kinh Te               3   8.533333
1  May Tinh               4   6.800000
2  Toan Tin               3   7.966667


## b, Tính tổng số sinh viên và điểm trung bình theo môn học:

In [9]:
query_avg_course = '''
SELECT c.course_name, COUNT(s.student_id) as total_students, AVG(s.score) as avg_score
FROM student s
JOIN course c ON s.course_id = c.id
GROUP BY c.course_name
'''
df_avg_course = pd.read_sql(query_avg_course, conn)
print("\nTổng số sinh viên và điểm trung bình theo môn học:")
print(df_avg_course)


Tổng số sinh viên và điểm trung bình theo môn học:
  course_name  total_students  avg_score
0   Giai tich               4        7.2
1    Thong ke               2        9.2


## c, Phân loại sinh viên :

In [10]:
conditions = [
    (df_student['score'] >= 9.0),
    (df_student['score'] >= 6.0) & (df_student['score'] < 9.0),
    (df_student['score'] < 6.0)
]
labels = ['Xuất sắc', 'Tốt', 'Kém']
df_student['rank'] = pd.cut(df_student['score'], bins=[0, 6, 9, 10], labels=labels, right=False)
print("\nPhân loại sinh viên:")
print(df_student[['name', 'score', 'rank']])


Phân loại sinh viên:
                name  score      rank
0  Nguyen Minh Hoang    6.7       Tốt
1       Tran Thi Lan    9.2       Kém
2       Pham Van Nam    7.9       Tốt
3     Le Thanh Huyen    7.2       Tốt
4        Vu Quoc Anh    8.0       Tốt
5     Dang Thuy Linh    5.5  Xuất sắc
6      Bui Tien Dung    9.2       Kém
7        Ho Ngoc Mai    8.8       Tốt
8     Duong Huu Phuc    7.2       Tốt
9       Cao Thi Hanh    7.0       Tốt


# 3. Xếp hạng sinh viên.

## a, Xếp hạng sinh viên theo điểm:

In [11]:
# Xếp hạng sinh viên theo điểm
print("\nXếp hạng theo điểm số:")
df_rank_score = df_student.sort_values(by="score", ascending=False)
print(df_rank_score.head(3))  # Top 3 sinh viên điểm cao nhất
print(df_rank_score.tail(3))  # Top 3 sinh viên điểm thấp nhất



Xếp hạng theo điểm số:
   student_id           name     class  course_id  score rank
1           2   Tran Thi Lan   Kinh Te       34.0    9.2  Kém
6           7  Bui Tien Dung   Kinh Te       34.0    9.2  Kém
7           8    Ho Ngoc Mai  Toan Tin       20.0    8.8  Tốt
   student_id               name     class  course_id  score      rank
9          10       Cao Thi Hanh  May Tinh        NaN    7.0       Tốt
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7       Tốt
5           6     Dang Thuy Linh  May Tinh       24.0    5.5  Xuất sắc


## b, Xếp hạng sinh viên theo điểm trong từng lớp:

In [12]:
print("\nXếp hạng theo điểm số trong từng lớp:")
df_rank_class = df_student.sort_values(by=["class", "score"], ascending=[True, False])
print(df_rank_class.groupby("class").head(3))  # Top 3 sinh viên điểm cao nhất từng lớp
print(df_rank_class.groupby("class").tail(3))  # Top 3 sinh viên điểm thấp nhất từng lớp



Xếp hạng theo điểm số trong từng lớp:
   student_id               name     class  course_id  score rank
1           2       Tran Thi Lan   Kinh Te       34.0    9.2  Kém
6           7      Bui Tien Dung   Kinh Te       34.0    9.2  Kém
8           9     Duong Huu Phuc   Kinh Te        NaN    7.2  Tốt
4           5        Vu Quoc Anh  May Tinh       24.0    8.0  Tốt
9          10       Cao Thi Hanh  May Tinh        NaN    7.0  Tốt
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7  Tốt
7           8        Ho Ngoc Mai  Toan Tin       20.0    8.8  Tốt
2           3       Pham Van Nam  Toan Tin        NaN    7.9  Tốt
3           4     Le Thanh Huyen  Toan Tin       20.0    7.2  Tốt
   student_id               name     class  course_id  score      rank
1           2       Tran Thi Lan   Kinh Te       34.0    9.2       Kém
6           7      Bui Tien Dung   Kinh Te       34.0    9.2       Kém
8           9     Duong Huu Phuc   Kinh Te        NaN    7.2       Tốt
9          10    

## c, Xếp hạng sinh viên theo điểm trong từng môn học:

In [13]:
print("\nXếp hạng theo điểm số trong từng môn học:")
df_rank_course = df_student.sort_values(by=["course_id", "score"], ascending=[True, False])
print(df_rank_course.groupby("course_id").head(3))  # Top 3 sinh viên điểm cao nhất từng môn học
print(df_rank_course.groupby("course_id").tail(3))  # Top 3 sinh viên điểm thấp nhất từng môn học



Xếp hạng theo điểm số trong từng môn học:
   student_id               name     class  course_id  score      rank
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7       Tốt
7           8        Ho Ngoc Mai  Toan Tin       20.0    8.8       Tốt
3           4     Le Thanh Huyen  Toan Tin       20.0    7.2       Tốt
4           5        Vu Quoc Anh  May Tinh       24.0    8.0       Tốt
5           6     Dang Thuy Linh  May Tinh       24.0    5.5  Xuất sắc
1           2       Tran Thi Lan   Kinh Te       34.0    9.2       Kém
6           7      Bui Tien Dung   Kinh Te       34.0    9.2       Kém
   student_id               name     class  course_id  score      rank
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7       Tốt
7           8        Ho Ngoc Mai  Toan Tin       20.0    8.8       Tốt
3           4     Le Thanh Huyen  Toan Tin       20.0    7.2       Tốt
4           5        Vu Quoc Anh  May Tinh       24.0    8.0       Tốt
5           6     Dang Thuy Linh  

# 4. Bổ sung thêm trường 'graduation_date'.

In [14]:
# Tính graduation_date
def calculate_graduation_date(score):
    current_date = datetime.now()
    if score >= 9.0:
        return current_date + timedelta(days=365*3)
    elif score >= 6.0:
        return current_date + timedelta(days=365*4)
    else:
        return current_date + timedelta(days=365*5)

df_student["graduation_date"] = df_student["score"].apply(calculate_graduation_date)
print(df_student)

   student_id               name     class  course_id  score      rank  \
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7       Tốt   
1           2       Tran Thi Lan   Kinh Te       34.0    9.2       Kém   
2           3       Pham Van Nam  Toan Tin        NaN    7.9       Tốt   
3           4     Le Thanh Huyen  Toan Tin       20.0    7.2       Tốt   
4           5        Vu Quoc Anh  May Tinh       24.0    8.0       Tốt   
5           6     Dang Thuy Linh  May Tinh       24.0    5.5  Xuất sắc   
6           7      Bui Tien Dung   Kinh Te       34.0    9.2       Kém   
7           8        Ho Ngoc Mai  Toan Tin       20.0    8.8       Tốt   
8           9     Duong Huu Phuc   Kinh Te        NaN    7.2       Tốt   
9          10       Cao Thi Hanh  May Tinh        NaN    7.0       Tốt   

             graduation_date  
0 2029-03-13 07:45:41.222260  
1 2028-03-13 07:45:41.222260  
2 2029-03-13 07:45:41.222260  
3 2029-03-13 07:45:41.222260  
4 2029-03-13 07:45:41.222260  


 **1. Đánh giá điểm số sinh viên**  
- **Phân bố điểm số:**  
  - Khoảng điểm dao động từ **5.5 đến 9.2**.  
  - Sinh viên có điểm cao nhất: **9.2** (có 2 sinh viên đạt).  
  - Sinh viên có điểm thấp nhất: **5.5**.  

- **Mức độ phân loại:**  
  - **Xuất sắc (>= 9.0):** 2 sinh viên  
  - **Giỏi (>= 8.0 & < 9.0):** 3 sinh viên  
  - **Khá (>= 7.0 & < 8.0):** 3 sinh viên  
  - **Trung bình (>= 6.0 & < 7.0):** 1 sinh viên  
  - **Yếu (< 6.0):** 1 sinh viên  

---

 **2. Xếp hạng sinh viên theo điểm số**  
- **Top 3 sinh viên có điểm cao nhất:**  
  - 1. **Tran Thi Lan** - 9.2  
  - 2. **Bui Tien Dung** - 9.2  
  - 3. **Ho Ngoc Mai** - 8.8  

- **Top 3 sinh viên có điểm thấp nhất:**  
  - 1. **Dang Thuy Linh** - 5.5  
  - 2. **Nguyen Minh Hoang** - 6.7  
  - 3. **Cao Thi Hanh** - 7.0  

---

 **3. Phân loại sinh viên theo lớp & môn học**  

 **Xếp hạng theo lớp:**  
- **Lớp "Kinh Tế":**  
  - Điểm cao nhất: **Tran Thi Lan - 9.2**  
  - Điểm thấp nhất: **Duong Huu Phuc - 7.2**  

- **Lớp "Toán Tin":**  
  - Điểm cao nhất: **Ho Ngoc Mai - 8.8**  
  - Điểm thấp nhất: **Le Thanh Huyen - 7.2**  

- **Lớp "Máy Tính":**  
  - Điểm cao nhất: **Vu Quoc Anh - 8.0**  
  - Điểm thấp nhất: **Dang Thuy Linh - 5.5**  

 **Xếp hạng theo môn học:**  
- **Môn "Giải Tích" (ID 12):**  
  - Không có dữ liệu sinh viên  

- **Môn "Thống Kê" (ID 34):**  
  - Điểm cao nhất: **Tran Thi Lan - 9.2**  
  - Điểm thấp nhất: **Bui Tien Dung - 9.2**  

- **Môn "Tin Học" (ID 26):**  
  - Không có dữ liệu sinh viên  

---

 **4. Tính ngày tốt nghiệp (Graduation Date)**  
- **Công thức:**  
  - Sinh viên xuất sắc (>=9.0): **Tốt nghiệp sau 3 năm**  
  - Sinh viên khá, giỏi (6.0 - 8.9): **Tốt nghiệp sau 4 năm**  
  - Sinh viên trung bình, yếu (<6.0): **Tốt nghiệp sau 5 năm**  

- **Ví dụ kết quả:**  
  - **Tran Thi Lan (9.2) → Tốt nghiệp: 2028**  
  - **Pham Van Nam (7.9) → Tốt nghiệp: 2029**  
  - **Dang Thuy Linh (5.5) → Tốt nghiệp: 2030**  

---

 **Kết luận**  
- **Tổng thể:** Lớp có sự phân bố điểm số khá đều, tuy nhiên có sự chênh lệch giữa các nhóm điểm cao và thấp.  
- **Sinh viên xuất sắc:** Có 2 sinh viên đạt điểm rất cao (9.2).  
- **Sinh viên cần cải thiện:** Một số sinh viên có điểm thấp dưới 6.0..  
- **Tình trạng tốt nghiệp:** Phần lớn sinh viên sẽ tốt nghiệp sau **4 năm**, một số sinh viên giỏi hơn có thể tốt nghiệp sớm hơn.  

---